# Problem Modification

**Originally Contributed by**: Arpit Bhatia

This tutorial deals with how to modify models after they have been created and solved.
This functionality can be useful, for example, 
when we are solving many similar models in succession or generating the model dynamically. 
Additionally it is sometimes desirable for the solver to re-start from the last solution to 
reduce running times for successive solves (“hot-start”).

In [1]:
using JuMP

## Modifying Variables

In [2]:
model = Model()
@variable(model, x);

### Variable Bounds

The `set_lower_bound` and `set_upper_bound` functions can be used to create as well as 
modify an existing variable bound.

In [3]:
set_lower_bound(x, 3)
lower_bound(x)

3.0

In [4]:
set_lower_bound(x, 2)
lower_bound(x)

2.0

We can delete variable bounds using the `delete_lower_bound` and `delete_upper_bound` functions.

In [5]:
delete_lower_bound(x)
has_lower_bound(x)

false

We can assign a fixed value to a variable using `fix`.

In [6]:
fix(x, 5)
fix_value(x)

5.0

However, fixing a variable with existing bounds will throw an error.

In [7]:
@variable(model, y >= 0);

In [8]:
fix(y, 2)

ErrorException: Unable to fix y to 2 because it has existing variable bounds. Consider calling `JuMP.fix(variable, value; force=true)` which will delete existing bounds before fixing the variable.

As we can see in the error message above, 
we have to specify to JuMP that we wish to forcefuly remove the bound.

In [9]:
fix(y, 2; force = true)
fix_value(y)

2.0

We can also call the `unfix` function to remove the fixed value.

In [10]:
unfix(x)
is_fixed(x)

false

### Deleting Variables

The `all_variables` function returns a list of all variables present in the model.

In [11]:
all_variables(model)

2-element Array{VariableRef,1}:
 x
 y

To delete variables from a model, we can use the `delete` function.

In [12]:
delete(model, x)
all_variables(model)

1-element Array{VariableRef,1}:
 y

We can also check whether a variable is a valid JuMP variable in a model using the `is_valid` function.

In [13]:
is_valid(model, x)

false

## Modifying Constraints

In [14]:
model = Model()
@variable(model, x);

### Modifying a Variable Coefficient
It is also possible to modify the scalar coefficients 
(but notably not yet the quadratic coefficients) using the `set_normalized_coefficient` function.

In [15]:
@constraint(model, con, 2x <= 1);

In [16]:
set_normalized_coefficient(con, x, 3)
con

con : 3 x ≤ 1.0

### Deleting a Constraint
Just like for deleting variables, we can use the `delete` function for constraints as well.

In [17]:
delete(model, con)
is_valid(model, con)

false

## Modifying the Objective

In [18]:
model = Model()
@variable(model, x)
@objective(model, Min, 7x + 4);

The function `objective_function` is used to query the objective of a model.

In [19]:
objective_function(model)

7 x + 4

`objective_sense` is similarily used to query the objective sense of a model.

In [20]:
objective_sense(model)

MIN_SENSE::OptimizationSense = 0

To easiest way to change the objective is to simply call `@objective` again 
- the previous objective function and sense will be replaced.

In [21]:
@objective(model, Max, 8x + 3)
objective_function(model)

8 x + 3

In [22]:
objective_sense(model)

MAX_SENSE::OptimizationSense = 1

Another way is to change the objective is to 
use the low-level functions `set_objective_function` and `set_objective_sense`.

In [23]:
set_objective_function(model, 5x + 11)
objective_function(model)

5 x + 11

In [24]:
set_objective_sense(model, MOI.MIN_SENSE) 
objective_sense(model)

MIN_SENSE::OptimizationSense = 0

Note that we can't use the Min and Max shortcuts here as `set_objective_sense` is a low level function.